# Vector Database Setup and Exploration

This notebook explores ChromaDB for our LLM long-term memory project.

## Goals:
- Install and setup ChromaDB
- Create a simple collection
- Add some sample data
- Test basic similarity search
- Understand how embeddings work

## What we're building:
A memory system that stores conversations and retrieves similar past interactions.